In [11]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# 1. CHARGEMENT DES DONNÉES
def load_data(file_path):
    """Charge les données depuis le fichier Excel"""
    df = pd.read_excel(file_path)
    print(f"✅ Données chargées : {df.shape[0]} lignes, {df.shape[1]} colonnes")
    return df

# 2. DÉTECTION DES VALEURS ABERRANTES
def detect_outliers(df, numerical_cols=None, method='iqr', threshold=1.5):
    """Détecte les valeurs aberrantes dans les colonnes numériques"""

    if numerical_cols is None:
        numerical_cols = ['speed_kmh', 'traffic_density', 'air_quality_index',
                         'latitude', 'longitude']

    outliers_info = {}
    outlier_indices = set()

    for col in numerical_cols:
        if col not in df.columns:
            continue

        data = df[col].dropna()

        if method == 'iqr':
            # Méthode IQR (Interquartile Range)
            Q1 = data.quantile(0.25)
            Q3 = data.quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - threshold * IQR
            upper_bound = Q3 + threshold * IQR

            outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]

        elif method == 'zscore':
            # Méthode Z-score
            z_scores = np.abs(stats.zscore(data))
            outliers = df[z_scores > threshold]

        elif method == 'percentile':
            # Méthode des percentiles
            lower_bound = data.quantile(0.01)
            upper_bound = data.quantile(0.99)
            outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]

        if len(outliers) > 0:
            outliers_info[col] = {
                'count': len(outliers),
                'percentage': (len(outliers) / len(df)) * 100,
                'indices': outliers.index.tolist(),
                'min_value': outliers[col].min(),
                'max_value': outliers[col].max()
            }
            outlier_indices.update(outliers.index)

    return outliers_info, list(outlier_indices)

# 3. TRAITEMENT DES VALEURS ABERRANTES
def handle_outliers(df, numerical_cols=None, method='winsorize', winsorize_limits=(0.01, 0.01)):
    """Traite les valeurs aberrantes selon différentes méthodes"""

    df_clean = df.copy()

    if numerical_cols is None:
        numerical_cols = ['speed_kmh', 'traffic_density', 'air_quality_index']

    print("\n🔍 TRAITEMENT DES VALEURS ABERRANTES")
    print("=" * 50)

    # Détection initiale
    outliers_info, outlier_indices = detect_outliers(df_clean, numerical_cols)

    if outliers_info:
        print(f"📊 {len(outlier_indices)} enregistrements avec valeurs aberrantes détectés")

        for col, info in outliers_info.items():
            print(f"\n  {col}:")
            print(f"    • {info['count']} outliers ({info['percentage']:.2f}%)")
            print(f"    • Plage des outliers: [{info['min_value']:.2f}, {info['max_value']:.2f}]")
            print(f"    • Plage normale: [{df_clean[col].min():.2f}, {df_clean[col].max():.2f}]")
    else:
        print("✅ Aucune valeur aberrante détectée")

    # Application du traitement selon la méthode choisie
    for col in numerical_cols:
        if col not in df_clean.columns:
            continue

        if method == 'winsorize':
            # Winsorization : remplace les extrêmes par des percentiles
            lower_limit = winsorize_limits[0]
            upper_limit = 1 - winsorize_limits[1]

            lower_bound = df_clean[col].quantile(lower_limit)
            upper_bound = df_clean[col].quantile(upper_limit)

            df_clean[col] = np.where(df_clean[col] < lower_bound, lower_bound, df_clean[col])
            df_clean[col] = np.where(df_clean[col] > upper_bound, upper_bound, df_clean[col])

            print(f"\n✅ {col}: Winsorization appliquée (limites: {lower_limit}, {upper_limit})")

        elif method == 'cap':
            # Capping avec IQR
            Q1 = df_clean[col].quantile(0.25)
            Q3 = df_clean[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            df_clean[col] = np.where(df_clean[col] < lower_bound, lower_bound, df_clean[col])
            df_clean[col] = np.where(df_clean[col] > upper_bound, upper_bound, df_clean[col])

            print(f"\n✅ {col}: Capping IQR appliqué")

        elif method == 'log':
            # Transformation logarithmique (pour données asymétriques)
            if (df_clean[col] > 0).all():
                df_clean[col] = np.log1p(df_clean[col])
                print(f"\n✅ {col}: Transformation logarithmique appliquée")
            else:
                print(f"\n⚠️  {col}: Transformation log impossible (valeurs négatives)")

        elif method == 'remove':
            # Suppression des outliers (méthode agressive)
            Q1 = df_clean[col].quantile(0.25)
            Q3 = df_clean[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            mask = (df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)
            df_clean = df_clean[mask]
            print(f"\n✅ {col}: Outliers supprimés")

    # Vérification après traitement
    outliers_info_after, _ = detect_outliers(df_clean, numerical_cols)
    if outliers_info_after:
        remaining_outliers = sum(info['count'] for info in outliers_info_after.values())
        print(f"\n⚠️  {remaining_outliers} outliers restants après traitement")
    else:
        print("\n🎉 Toutes les valeurs aberrantes ont été traitées")

    return df_clean

# 4. NETTOYAGE DES DONNÉES
def clean_data(df, outlier_method='winsorize'):
    """Nettoie les données avec traitement des outliers"""
    df_clean = df.copy()

    # Conversion du timestamp
    df_clean['timestamp'] = pd.to_datetime(df_clean['timestamp'])

    # Extraction des caractéristiques temporelles
    df_clean['hour'] = df_clean['timestamp'].dt.hour
    df_clean['day_of_week'] = df_clean['timestamp'].dt.dayofweek
    df_clean['month'] = df_clean['timestamp'].dt.month
    df_clean['is_weekend'] = df_clean['day_of_week'].isin([5, 6]).astype(int)

    # Vérification des valeurs manquantes
    print("\n🔍 Valeurs manquantes par colonne :")
    missing = df_clean.isnull().sum()
    print(missing[missing > 0] if missing.sum() > 0 else "✅ Aucune valeur manquante")

    # Imputation des valeurs manquantes
    if missing.sum() > 0:
        for col in df_clean.columns:
            if df_clean[col].isnull().sum() > 0:
                if df_clean[col].dtype in ['float64', 'int64']:
                    df_clean[col].fillna(df_clean[col].median(), inplace=True)
                else:
                    df_clean[col].fillna(df_clean[col].mode()[0], inplace=True)

    # Traitement des valeurs aberrantes
    df_clean = handle_outliers(df_clean, method=outlier_method)

    # Suppression des doublons
    initial_rows = len(df_clean)
    df_clean = df_clean.drop_duplicates()
    removed_duplicates = initial_rows - len(df_clean)
    if removed_duplicates > 0:
        print(f"\n📊 {removed_duplicates} doublons supprimés")

    return df_clean

# 5. TRANSFORMATION DES DONNÉES
def transform_data(df):
    """Transforme les données pour l'analyse"""
    df_transformed = df.copy()

    # Catégorisation des variables
    def categorize_aqi(aqi):
        if aqi <= 50:
            return 'Bon'
        elif aqi <= 100:
            return 'Modéré'
        elif aqi <= 150:
            return 'Mauvais'
        else:
            return 'Dangereux'

    df_transformed['aqi_category'] = df_transformed['air_quality_index'].apply(categorize_aqi)

    def categorize_speed(speed):
        if speed <= 20:
            return 'Lente'
        elif speed <= 35:
            return 'Normale'
        else:
            return 'Rapide'

    df_transformed['speed_category'] = df_transformed['speed_kmh'].apply(categorize_speed)

    def categorize_traffic(density):
        if density <= 0.25:
            return 'Fluide'
        elif density <= 0.5:
            return 'Modéré'
        else:
            return 'Dense'

    df_transformed['traffic_category'] = df_transformed['traffic_density'].apply(categorize_traffic)

    # Encodage
    le = LabelEncoder()
    df_transformed['weather_encoded'] = le.fit_transform(df_transformed['weather'])

    return df_transformed

# 6. CRÉATION DE FEATURES
def create_features(df):
    """Crée de nouvelles features"""
    df_features = df.copy()

    df_features['speed_traffic_product'] = df_features['speed_kmh'] * df_features['traffic_density']
    df_features['traffic_aqi_flag'] = ((df_features['traffic_density'] < 0.2) & 
                                      (df_features['air_quality_index'] > 70)).astype(int)

    # Heures de pointe
    def is_rush_hour(hour):
        return 1 if (7 <= hour <= 9) or (17 <= hour <= 19) else 0

    df_features['is_rush_hour'] = df_features['hour'].apply(is_rush_hour)

    # Moment de la journée
    def time_of_day(hour):
        if 5 <= hour < 12:
            return 'Matin'
        elif 12 <= hour < 17:
            return 'Après-midi'
        elif 17 <= hour < 22:
            return 'Soir'
        else:
            return 'Nuit'

    df_features['time_of_day'] = df_features['hour'].apply(time_of_day)



    return df_features

# 7. PIPELINE ML AVEC ROBUSTSCALER POUR OUTLIERS
def create_ml_pipeline(outlier_robust=True):
    """Crée un pipeline ML robuste aux outliers"""

    numeric_features = ['speed_kmh', 'traffic_density', 'air_quality_index',
                       'latitude', 'longitude', 'hour', 'speed_traffic_product']
    categorical_features = ['weather', 'aqi_category', 'speed_category',
                           'traffic_category', 'time_of_day']

    # Utilisation de RobustScaler pour les outliers
    if outlier_robust:
        numeric_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),  # Median plus robuste
            ('scaler', RobustScaler())  # Meilleur pour les outliers que StandardScaler
        ])
    else:
        numeric_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', LabelEncoder())
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ])

    return preprocessor

# 8. ANALYSE DES OUTLIERS DÉTAILLÉE
def detailed_outlier_analysis(df):
    """Analyse détaillée des valeurs aberrantes"""

    numerical_cols = ['speed_kmh', 'traffic_density', 'air_quality_index', 'speed_traffic_product']

    print("\n📊 ANALYSE DÉTAILLÉE DES VALEURS ABERRANTES")
    print("=" * 60)

    for col in numerical_cols:
        if col in df.columns:
            data = df[col].dropna()

            # Statistiques
            stats_dict = {
                'Moyenne': data.mean(),
                'Médiane': data.median(),
                'Std': data.std(),
                'Min': data.min(),
                'Max': data.max(),
                'Q1': data.quantile(0.25),
                'Q3': data.quantile(0.75),
                'IQR': data.quantile(0.75) - data.quantile(0.25),
                'Skewness': data.skew(),
                'Kurtosis': data.kurtosis()
            }

            print(f"\n📈 {col}:")
            for key, value in stats_dict.items():
                print(f"  {key}: {value:.4f}")

            # Détection IQR
            Q1 = stats_dict['Q1']
            Q3 = stats_dict['Q3']
            IQR = stats_dict['IQR']
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
            outlier_percentage = (len(outliers) / len(df)) * 100

            print(f"  Outliers (IQR): {len(outliers)} ({outlier_percentage:.2f}%)")
            print(f"  Plage normale: [{lower_bound:.4f}, {upper_bound:.4f}]")

            # Visualisation textuelle
            if len(outliers) > 0:
                print(f"  Exemples d'outliers: {outliers[col].head(5).values}")

# 9. PIPELINE COMPLET
def run_full_pipeline(file_path, outlier_method='winsorize', outlier_robust=True):
    """Exécute le pipeline complet avec traitement des outliers"""

    print("🚀 DÉMARRAGE DU PIPELINE AVEC TRAITEMENT DES OUTLIERS")
    print("=" * 70)
    print(f"📌 Méthode de traitement des outliers: {outlier_method}")
    print(f"📌 RobustScaler pour ML: {outlier_robust}")

    # Étape 1: Chargement
    df = load_data(file_path)

    # Étape 2: Analyse initiale des outliers
    print("\n🔍 ANALYSE INITIALE DES OUTLIERS")
    detailed_outlier_analysis(df)

    # Étape 3: Nettoyage avec traitement des outliers
    df = clean_data(df, outlier_method=outlier_method)

    # Étape 4: Transformation
    df = transform_data(df)

    # Étape 5: Création de features
    df = create_features(df)

    # Étape 6: Analyse après traitement
    print("\n🔍 ANALYSE APRÈS TRAITEMENT DES OUTLIERS")
    detailed_outlier_analysis(df)

    # Étape 7: Pipeline ML robuste
    preprocessor = create_ml_pipeline(outlier_robust=outlier_robust)

    print("\n" + "=" * 70)
    print("✅ PIPELINE TERMINÉ AVEC SUCCÈS")
    print(f"📋 Données finales : {df.shape[0]} lignes, {df.shape[1]} colonnes")

    return df, preprocessor

# 10. EXPORT DES RÉSULTATS
def export_results(df, output_path='processed_mobility_data_with_outliers.csv'):
    """Exporte les données traitées"""
    df.to_csv(output_path, index=False)
    print(f"\n💾 Données exportées vers : {output_path}")

# 11. EXÉCUTION AVEC OPTIONS
if __name__ == "__main__":
    INPUT_FILE = "C:/Users/PC/Desktop/Bootcamp_FN/mobility_urban_pollution_300.xlsx"

    # Options de traitement des outliers
    METHODS = {
        'winsorize': 'Winsorization (remplacement par percentiles)',
        'cap': 'Capping IQR (troncature des extrêmes)',
        'log': 'Transformation logarithmique',
        'remove': 'Suppression des outliers'
    }

    print("🎯 OPTIONS DE TRAITEMENT DES OUTLIERS")
    print("=" * 50)
    for key, value in METHODS.items():
        print(f"  {key}: {value}")

    # Choix de la méthode (vous pouvez le rendre interactif)
    chosen_method = 'winsorize'  # Par défaut
    # Pour rendre interactif : chosen_method = input("\nChoisissez une méthode: ")

    try:
        # Exécution avec la méthode choisie
        processed_data, ml_pipeline = run_full_pipeline(
            INPUT_FILE,
            outlier_method=chosen_method,
            outlier_robust=True
        )

        # Affichage d'échantillon
        print("\n📄 Échantillon des données traitées :")
        print(processed_data[['speed_kmh', 'traffic_density', 'air_quality_index',
                              'weather', 'aqi_category']].head())

        # Export
        export_results(processed_data, f"mobility_data_processed_{chosen_method}.csv")

        print(f"\n🛠️ Pipeline ML créé avec RobustScaler: {ml_pipeline}")

    except Exception as e:
        print(f"\n❌ Erreur : {e}")
        import traceback
        traceback.print_exc()

🎯 OPTIONS DE TRAITEMENT DES OUTLIERS
  winsorize: Winsorization (remplacement par percentiles)
  cap: Capping IQR (troncature des extrêmes)
  log: Transformation logarithmique
  remove: Suppression des outliers
🚀 DÉMARRAGE DU PIPELINE AVEC TRAITEMENT DES OUTLIERS
📌 Méthode de traitement des outliers: winsorize
📌 RobustScaler pour ML: True
✅ Données chargées : 300 lignes, 8 colonnes

🔍 ANALYSE INITIALE DES OUTLIERS

📊 ANALYSE DÉTAILLÉE DES VALEURS ABERRANTES

📈 speed_kmh:
  Moyenne: 28.7867
  Médiane: 28.5000
  Std: 7.4039
  Min: 10.0000
  Max: 48.0000
  Q1: 24.0000
  Q3: 33.0000
  IQR: 9.0000
  Skewness: 0.0295
  Kurtosis: -0.1488
  Outliers (IQR): 4 (1.33%)
  Plage normale: [10.5000, 46.5000]
  Exemples d'outliers: [47 10 48 47]

📈 traffic_density:
  Moyenne: 0.3114
  Médiane: 0.2850
  Std: 0.1687
  Min: 0.0500
  Max: 0.8400
  Q1: 0.1800
  Q3: 0.4300
  IQR: 0.2500
  Skewness: 0.6195
  Kurtosis: -0.0819
  Outliers (IQR): 1 (0.33%)
  Plage normale: [-0.1950, 0.8050]
  Exemples d'outlier

In [13]:
processed_data.head()

,route_id,timestamp,latitude,longitude,speed_kmh,traffic_density,air_quality_index,weather,hour,day_of_week,month,is_weekend,aqi_category,speed_category,traffic_category,weather_encoded,speed_traffic_product,traffic_aqi_flag,is_rush_hour,time_of_day
0,R004,2025-08-01 13:17:05,14.69869,-17.35986,42.0,0.18,45.0,Rain,13,4,8,0,Bon,Rapide,Fluide,2,7.56,0,0,Après-midi
1,R008,2025-08-03 23:00:53,14.67364,-17.51332,24.0,0.54,80.0,Sunny,23,6,8,1,Modéré,Normale,Dense,3,12.96,0,0,Nuit
2,R004,2025-08-07 20:50:58,14.68798,-17.47673,25.0,0.39,66.0,Sunny,20,3,8,0,Modéré,Normale,Modéré,3,9.75,0,0,Soir
3,R014,2025-08-01 15:15:26,14.65846,-17.36022,32.0,0.33,75.0,Sunny,15,4,8,0,Modéré,Normale,Modéré,3,10.56,0,0,Après-midi
4,R007,2025-08-04 01:46:02,14.66586,-17.45096,21.0,0.43,71.0,Cloudy,1,0,8,0,Modéré,Normale,Modéré,0,9.03,0,0,Nuit


In [23]:
!pip install pymysql

Defaulting to user installation because normal site-packages is not writeable


In [24]:
from sqlalchemy import create_engine
# Connexion à MySQL local
def connect_to_mysql():
    """Établit la connexion à MySQL"""
    try:
        engine = create_engine("mysql+pymysql://root@localhost/mobility_db")
        connection = engine.connect()
        print("✅ Connecté à MySQL avec succès")
        return engine, connection
    except Exception as e:
        print(f"❌ Erreur de connexion: {e}")
        return None, None

# Tester la connexion
engine, conn = connect_to_mysql()

✅ Connecté à MySQL avec succès


In [27]:
def save_to_existing_table(df, table_name='mobility_processed'):
    """Insère dans la table existante avec mapping des colonnes"""
    
    # Vérifier les colonnes
    print("📋 Colonnes disponibles dans vos données:")
    print(df.columns.tolist())
    
    # Mapping entre vos colonnes et la table
    column_mapping = {
        'route_id': 'route_id',
        'timestamp': 'timestamp',
        'latitude': 'latitude',
        'longitude': 'longitude',
        'speed_kmh': 'speed_kmh',
        'traffic_density': 'traffic_density',
        'air_quality_index': 'air_quality_index',
        'weather': 'weather',
        'hour': 'hour',
        'day_of_week': 'day_of_week',
        'month': 'month',
        'is_weekend': 'is_weekend',
        'aqi_category': 'aqi_category',
        'speed_category': 'speed_category',
        'traffic_category': 'traffic_category',
        'weather_encoded': 'weather_encoded',
        'speed_traffic_product': 'speed_traffic_product',
        'traffic_aqi_flag': 'traffic_aqi_flag',
        'is_rush_hour': 'is_rush_hour',
        'time_of_day': 'time_of_day'
        # 'created_at' sera auto-généré
    }
    
    # Sélectionner et renommer les colonnes
    df_to_insert = df[list(column_mapping.keys())].rename(columns=column_mapping)
    
    # Connexion MySQL
    engine = create_engine("mysql+pymysql://root@localhost/mobility_db")
    
    # Insérer avec append (ne pas remplacer la table)
    df_to_insert.to_sql(table_name, 
                       engine, 
                       if_exists='append',  # ← CRUCIAL: 'append' pas 'replace'
                       index=False)
    
    # Vérifier
    count = pd.read_sql(f"SELECT COUNT(*) as count FROM {table_name}", engine)
    print(f"✅ {len(df_to_insert)} lignes insérées")
    print(f"📊 Total dans la table: {count['count'][0]} lignes")
    
    engine.dispose()
    return True

# Utilisation
save_to_existing_table(processed_data)

📋 Colonnes disponibles dans vos données:
['route_id', 'timestamp', 'latitude', 'longitude', 'speed_kmh', 'traffic_density', 'air_quality_index', 'weather', 'hour', 'day_of_week', 'month', 'is_weekend', 'aqi_category', 'speed_category', 'traffic_category', 'weather_encoded', 'speed_traffic_product', 'traffic_aqi_flag', 'is_rush_hour', 'time_of_day']
✅ 300 lignes insérées
📊 Total dans la table: 300 lignes


True

In [29]:
query = "SELECT * FROM mobility_processed LIMIT 10"
df_result = pd.read_sql(query, engine)
df_result.head()

,id,route_id,timestamp,latitude,longitude,speed_kmh,traffic_density,air_quality_index,weather,hour,...,is_weekend,aqi_category,speed_category,traffic_category,weather_encoded,speed_traffic_product,traffic_aqi_flag,is_rush_hour,time_of_day,created_at
0,1,R004,2025-08-01 13:17:05,14.69869,-17.35986,42.0,0.18,45,Rain,13,...,0,Bon,Rapide,Fluide,2,7.56,0,0,Après-midi,2026-01-16 21:30:06
1,2,R008,2025-08-03 23:00:53,14.67364,-17.51332,24.0,0.54,80,Sunny,23,...,1,Modéré,Normale,Dense,3,12.96,0,0,Nuit,2026-01-16 21:30:06
2,3,R004,2025-08-07 20:50:58,14.68798,-17.47673,25.0,0.39,66,Sunny,20,...,0,Modéré,Normale,Modéré,3,9.75,0,0,Soir,2026-01-16 21:30:06
3,4,R014,2025-08-01 15:15:26,14.65846,-17.36022,32.0,0.33,75,Sunny,15,...,0,Modéré,Normale,Modéré,3,10.56,0,0,Après-midi,2026-01-16 21:30:06
4,5,R007,2025-08-04 01:46:02,14.66586,-17.45096,21.0,0.43,71,Cloudy,1,...,0,Modéré,Normale,Modéré,0,9.03,0,0,Nuit,2026-01-16 21:30:06
